In [12]:

import pathlib
import hydra
import os, sys
import torch
sys.path.append('../')

import torch
import wandb
import hydra
import omegaconf
from omegaconf import DictConfig
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.utilities.warnings import PossibleUserWarning

from dgd import utils
from dgd.datasets.frag_dataset import FragDataset, FragDataModule, FragDatasetInfos
from dgd.analysis.graph_to_mol import PyGGraphToMolConverter
from dgd.datasets.frag_dataset import FRAG_GRAPH_FILE, FRAG_INDEX_FILE, FRAG_EDGE_FILE



In [14]:
config_dir = pathlib.Path('../configs/')
hydra.core.global_hydra.GlobalHydra.instance().clear()
hydra.initialize(config_dir)
cfg = hydra.compose('config.yaml')
dataset_config = cfg["dataset"] 
print(cfg)

{'general': {'name': 'graph-tf-model', 'wandb': 'online', 'gpus': 1, 'resume': None, 'test_only': None, 'check_val_every_n_epochs': 5, 'sample_every_val': 4, 'val_check_interval': None, 'samples_to_generate': 512, 'samples_to_save': 20, 'chains_to_save': 1, 'log_every_steps': 50, 'number_chain_steps': 50, 'final_model_samples_to_generate': 10000, 'final_model_samples_to_save': 30, 'final_model_chains_to_save': 20, 'evaluate_all_checkpoints': False}, 'model': {'type': 'discrete', 'transition': 'marginal', 'model': 'graph_tf', 'diffusion_steps': 500, 'diffusion_noise_schedule': 'cosine', 'n_layers': 5, 'extra_features': 'all', 'hidden_mlp_dims': {'X': 256, 'E': 128, 'y': 128}, 'hidden_dims': {'dx': 256, 'de': 64, 'dy': 64, 'n_head': 8, 'dim_ffX': 256, 'dim_ffE': 128, 'dim_ffy': 128}, 'lambda_train': [5, 0]}, 'train': {'n_epochs': 1000, 'batch_size': 512, 'lr': 0.0002, 'clip_grad': None, 'save_model': True, 'num_workers': 0, 'ema_decay': 0, 'progress_bar': False, 'weight_decay': 1e-12, 'o

/var/folders/9q/slm5n1dd2f524zmlmd4y58gh0000gn/T/ipykernel_93251/758915753.py:3: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  hydra.initialize(config_dir)


In [17]:

dataset = FragDataset(FRAG_GRAPH_FILE)
datamodule = FragDataModule(cfg)
#sampling_metrics = SBMSamplingMetrics(datamodule.dataloaders)

dataset_infos = FragDatasetInfos(datamodule, dataset_config)
converter = PyGGraphToMolConverter('../data/' + FRAG_INDEX_FILE, '../data/' +FRAG_EDGE_FILE)
#converter.graph_to_mol(graph)

Dataset /Users/daniellevy/Documents/courses/760/project/FragDiffusion/dgd/datasets/../../data/frag/mol_frag_graphs.pt loaded from file
Dataset /Users/daniellevy/Documents/courses/760/project/FragDiffusion/dgd/datasets/../../data/frag/mol_frag_graphs.pt loaded from file
Dataset sizes: train 8914, val 2229, test 2786


In [18]:
graph = dataset[0]
converter.graph_to_mol(graph, count_non_edge=True)

TypeError: PyGGraphToMolConverter.graph_to_mol() got an unexpected keyword argument 'count_non_edge'

In [1]:

#train_metrics = TrainAbstractMetricsDiscrete() if cfg.model.type == 'discrete' else TrainAbstractMetrics()
#visualization_tools = NonMolecularVisualization()

#if cfg.model.type == 'discrete' and cfg.model.extra_features is not None:
#    extra_features = ExtraFeatures(cfg.model.extra_features, dataset_info=dataset_infos)
#else:
extra_features = DummyExtraFeatures()
domain_features = DummyExtraFeatures()

dataset_infos.compute_input_output_dims(datamodule=datamodule, extra_features=extra_features,
                                        domain_features=domain_features)

model_kwargs = {'dataset_infos': dataset_infos, 'train_metrics': train_metrics,
                'sampling_metrics': sampling_metrics, 'visualization_tools': visualization_tools,
                'extra_features': extra_features, 'domain_features': domain_features}

: 

: 